In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, TickerData
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta

In [2]:
# obj = Candle.objects.filter(symbol__symbol="ambujacem", candle_type="M5", date__range=[datetime.strptime("17/10/2019",'%d/%m/%Y').date(), datetime.strptime("19/10/2019",'%d/%m/%Y').date()]).values()
# # # new_header = ["Exchange", "Token", "NA", "Symbol", "Full Name", "closed_price", "NA1", "NA2", "Tick Size", "Lot Size", "Instrument Type", "ISIN"]

# df = pd.DataFrame(list(obj))
# # # obj.columns = new_header
# # # new_obj = obj[(obj.close_price >= 55) & (obj.close_price <= 250)]
# symbol = Symbol.objects.get(symbol="pnb")
# # symbol.is_stock_ohl(date="17/10/2019")
# first_candle_price = df.iloc[0:]
# # previouse_day_last_candle_price = df.loc[:, ["open_price", "high_price", "low_price", "close_price"]]
# # min(current_candle_price.min().values)
# last_day_candles = df[(df.date <= datetime.strptime("18/10/2019",'%d/%m/%Y').date())]
# last_day_closing_price = last_day_candles.iloc[[-1]].close_price
# last_day_opening_price = last_day_candles.iloc[[0]].open_price
# today_open_price = df[(df.date >= datetime.strptime("18/10/2019",'%d/%m/%Y').date())].iloc[[0]].open_price
# # print(first_candle_price)
# today_open_price > 100


In [3]:
from ta.trend import macd, macd_diff, macd_signal, ema, ema_indicator
from ta.momentum import stoch, stoch_signal
stock = Symbol.objects.get(symbol="ibulhsgfin")
obj = stock.get_stock_data(days=1, end_date=datetime.strptime("30/10/2019",'%d/%m/%Y').date())
df = pd.DataFrame(list(obj.values()))
# df["macd"] = macd(df.close_price)
# df["macd_signal"] = macd_signal(df.close_price)
# df["diff"] = macd_diff(df.close_price)
# df.to_csv("stoch_analytics.csv")
# df.head(2)
# obj.values()
from django.core.cache import caches, cache

In [25]:
my_dict = {'timestamp': '1573630000000','id': 707117, 'symbol': "ibulhsgfin", 'candle_type': 'M5', 'open': 189.85, 'high': 190.0, 'low': 186.2, 'close': 189.45, 'volume': 979559, 'vtt': None, 'atp': None, 'total_buy_quantity': None, 'total_sell_quantity': None, 'lower_circuit': None, 'upper_circuit': None, 'bids': {}, 'asks': {}, 'date': datetime(2019, 10, 29, 9, 15), 'created_at': datetime(2019, 11, 1, 16, 56, 54, 210554), 'modified_at': datetime(2019, 11, 1, 10, 3, 16, 762851)}
data = [my_dict for i in range(0,20)]
# caches["redis"].set("my_cache", data)
def get_stock_current_candle(symbol:str):
    redis_cache = caches["redis"]
    cached_data = redis_cache.get(symbol)
    first_ticker = cached_data[0]
    current_ticker = cached_data[-1]
    df_ticker = {
#         "symbol": first_ticker.get("symbol").lower(),
        "candle_type": "M5",
        "open_price": first_ticker.get("open"),
        "high_price": max(first_ticker.get("high"), current_ticker.get("high")),
        "close_price": current_ticker.get("low"),
        "low_price": min(first_ticker.get("low"), current_ticker.get("low")),
        "volume": current_ticker.get("vtt"),
#         "atp": current_ticker.get("atp"),
        "total_buy_quantity": current_ticker.get("total_buy_quantity"),
        "total_sell_quantity": current_ticker.get("total_sell_quantity"),
#         "lower_circuit": current_ticker.get("lower_circuit"),
#         "upper_circuit": current_ticker.get("upper_circuit"),
#         "bids": {},
#         "asks": {},
        "date": datetime.fromtimestamp(int(current_ticker.get("timestamp")[:10]))
    }
    return df_ticker


# def get_macd_crossover(stock_name:str):
symbol = Symbol.objects.get(symbol="ibulhsgfin")
stock_data = symbol.get_stock_data(days=1, end_date=datetime.strptime("30/10/2019",'%d/%m/%Y').date())
current_candle_data = get_stock_current_candle("my_cache")
df1 = pd.DataFrame(list(stock_data.values("candle_type", "open_price", "high_price", "low_price", "close_price", "volume", "total_buy_quantity", "total_sell_quantity", "date")))
df2 = pd.DataFrame(current_candle_data, index=[0])
df = pd.concat([df1, df2], ignore_index=True)
df["macd"] = macd(df.close_price)
df["macd_signal"] = macd_signal(df.close_price)
df["macd_diff"] = macd_signal(df.close_price)
df


/Users/sonu2/Desktop/Sonu/env/lib/python3.7/site-packages/ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,candle_type,close_price,date,high_price,low_price,open_price,total_buy_quantity,total_sell_quantity,volume,macd,macd_signal,macd_diff
0,M5,189.45,2019-10-29 09:15:00,190.00,186.20,189.85,None,None,979559,NaN,NaN,NaN
1,M5,195.00,2019-10-29 09:20:00,195.45,189.20,189.40,None,None,1509329,NaN,NaN,NaN
2,M5,193.45,2019-10-29 09:25:00,196.00,193.25,195.10,None,None,1057051,NaN,NaN,NaN
3,M5,193.35,2019-10-29 09:30:00,193.50,191.50,193.25,None,None,694505,NaN,NaN,NaN
4,M5,193.30,2019-10-29 09:35:00,194.20,192.95,193.15,None,None,417329,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
146,M5,213.90,2019-10-30 15:10:00,215.75,212.45,215.65,None,None,1102214,-0.827429,-0.592115,-0.592115
147,M5,214.00,2019-10-30 15:15:00,215.50,212.80,213.85,None,None,1145645,-0.913249,-0.656342,-0.656342
148,M5,214.10,2019-10-30 15:20:00,215.00,213.50,214.00,None,None,554387,-0.962103,-0.717494,-0.717494
149,M5,213.75,2019-10-30 15:25:00,214.10,212.85,214.00,None,None,498739,-1.017334,-0.777462,-0.777462


In [11]:
from django.db.models import Max, Min, Sum
obj = SortedStocksList.objects.get(symbol__symbol="ibulhsgfin")
timestamp = obj.timestamps.all()
timestamp.aggregate(Sum("indicator__value"))

{'indicator__value__sum': 17}

In [6]:
# Process for Tasks

# 1. Socket Send data to function - Done
# 2. func store data in redis as list inside dict - 